<a href="https://colab.research.google.com/github/SSAI-Remote-Proctor/android_yolo/blob/master/yolov5_objectdetector_Ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




# Google Drive에 연결



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# YOLOv5 개발환경 세팅

In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard 886f1c03d839575afecb059accf74296fad395b6

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
HEAD is now at 886f1c0 DDP after autoanchor reorder (#2421)


In [ ]:
# install dependencies as necessary
!pip install tensorflow-addons
!pip install -qr requirements.txt  # install dependencies (ignore errors)

import torch

from IPython.display import Image, clear_output  # to display images
from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


# COCO dataset custom

In [ ]:
from IPython.display import Image


# Object detection with detec.py(--source 해당 파일)

In [ ]:
%%time
!python detect.py --weights yolov5l.pt --img 1280 --conf 0.4 --source ./detection_ex3.mp4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', exist_ok=False, img_size=1280, iou_thres=0.45, name='exp', project='runs/detect', save_conf=False, save_txt=False, source='./detection_ex3.mp4', update=False, view_img=False, weights=['yolov5l.pt'])
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 392 layers, 47025981 parameters, 0 gradients, 115.4 GFLOPS
video 1/1 (1/329) /content/yolov5/detection_ex3.mp4: 736x1280 1 cup, 2 chairs, 1 laptop, 1 cell phone, Done. (0.064s)
video 1/1 (2/329) /content/yolov5/detection_ex3.mp4: 736x1280 1 cup, 2 chairs, 1 laptop, 1 cell phone, Done. (0.046s)
video 1/1 (3/329) /content/yolov5/detection_ex3.mp4: 736x1280 1 cup, 2 chairs, 1 laptop, 1 cell phone, Done. (0.037s)
video 1/1 (4/329) /content/yolov5/detection_ex3.mp4: 736x1280 1 cup, 2 chairs, 1 laptop, 1 cell phone, Done. (0.037s)
video 1/1 (5/329) /content/yolov5/detection_ex3.mp4: 736x1280 1 cup, 2 chairs

# 결과 파일 다운로드

In [ ]:
from google.colab import files
files.download('./runs/detect/exp2/detection_ex3.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

참조 : https://lv99.tistory.com/73

# pb file을 Tflite로 변환

In [ ]:
%%bash
pip install onnx tensorflow-addons
git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

fatal: destination path 'onnx-tensorflow' already exists and is not an empty directory.


In [ ]:
import sys
sys.path.append("./onnx-tensorflow")

In [ ]:
#pt to onnx
!python models/export.py --weights ./weights/yolov5s.pt --img 640 --batch 1

Namespace(batch_size=1, device='cpu', dynamic=False, grid=False, img_size=[640, 640], weights='./weights/yolov5s.pt')
YOLOv5 v4.0-126-g886f1c0 torch 1.8.1+cu101 CPU

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients, 17.0 GFLOPS

Starting TorchScript export with torch 1.8.1+cu101...
/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:940: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  _force_outplace,
TorchScript export success, saved as ./weights/yolov5s.torchscript.pt

Starting ONNX export with onnx 1.9.0...
ONNX export success, saved as ./weights/yolov5s.onnx
CoreML export 

In [ ]:
#onnx to pb
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("./weights/yolov5s.onnx") #load onnx model
tf_rep = prepare(onnx_model) #prepare tf representation
tf_rep.export_graph("./output/tf_model.pb")

TypeError: ignored

In [ ]:
import tensorflow.compat.v1 as tf

saved_model_dir = "./output/yolov5s.pb"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops =[tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

fo = open("./output/yolov5s.tflite", "wb")
fo.write(tflite_model)
fo.close

INFO:tensorflow:Restoring parameters from ./output/yolov5s.pb/variables/variables


INFO:tensorflow:Restoring parameters from ./output/yolov5s.pb/variables/variables


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:The given SavedModel MetaGraphDef contains SignatureDefs with the following keys: {'serving_default', '__saved_model_init_op'}


INFO:tensorflow:input tensors info: 


...

INFO:tensorflow: tensor name: StatefulPartitionedCall:1, shape: (1, 3, 20, 20, 85), type: DT_FLOAT


INFO:tensorflow: tensor name: StatefulPartitionedCall:1, shape: (1, 3, 20, 20, 85), type: DT_FLOAT


INFO:tensorflow:Restoring parameters from ./output/yolov5s.pb/variables/variables


INFO:tensorflow:Restoring parameters from ./output/yolov5s.pb/variables/variables
Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or '@defun' to decorate the function.Use '@tf.function' or 

ConverterError: ignored